In [1]:
import pandas as pd
import random
import hashlib
from pymongo import MongoClient
import os
from faker import Faker

In [3]:
SERVICE_DB_CONN_STR = os.environ.get('ML_CLUSTER_CONN_STR')
mongo_client = MongoClient(SERVICE_DB_CONN_STR)

In [4]:
from datetime import datetime 

transactions = mongo_client.flashprd_12.transactions.find(
        {
            "type": "withdrawal",
            "recipient.isCompany": False,
            "sender.isCompany": False,
            "createdAt": {
                "$gte": datetime(2023, 6, 15),
                # "$lte": datetime(2023, 9, 15),
            },
        },
        {
            "amount": 1,
            "aml.paymentCheck": 1,
            "aml.recommendation": 1,
            "status": 1,
            "recipient.firstName": 1,
            "recipient.lastName": 1,
            "recipient.accountNo": 1,
            "recipient.bsb": 1,
            "createdAt": 1,
            "sender.firstName": 1,
            "sender.lastName": 1,
            "sender.accountNo": 1,
            }
        )
df = pd.DataFrame(list(transactions))

In [5]:
df.shape

(144496, 7)

In [6]:
paymentCheck = mongo_client.flashprd_12.paymentchecks.find(
        {
            "createdAt": {
                "$gte": datetime(2023, 6, 15),
                # "$lte": datetime(2023, 9, 16),
            },
        },
            {
                "fromLegActor.actorType": 1,
                "toLegActor.actorType": 1,
            }
        )
paymentCheck_df = pd.DataFrame(list(paymentCheck))

In [7]:
paymentCheck_df

,_id,fromLegActor,toLegActor
0,648a54d19fd4430fc27a900f,{'actorType': 'UNDECIDED'},NaN
1,648a552a43e60d0f62a5dc81,NaN,NaN
2,648a554543e60d5b82a5dcc7,{'actorType': 'GOOD'},{'actorType': 'GOOD'}
3,648a56ba9fd44305647a9903,{'actorType': 'GOOD'},{'actorType': 'GOOD'}
4,648a56bb9fd4430b657a9914,{'actorType': 'GOOD'},{'actorType': 'GOOD'}
...,...,...,...
254070,6561f740b3dd10043b5d9f98,NaN,NaN
254071,6561f897b3dd10b5085da423,NaN,NaN
254072,6561f92f8d6bf6cce61dc44b,{'actorType': 'GOOD'},{'actorType': 'GOOD'}
254073,6561fa3eb3dd10335b5dad00,NaN,NaN


In [8]:
recipient = pd.json_normalize(df['recipient']).rename(columns=lambda x: 'recipient.' + x)
df = pd.concat([df, recipient], axis=1)

sender = pd.json_normalize(df['sender']).rename(columns=lambda x: 'sender.' + x)
df = pd.concat([df, sender], axis=1)

aml = pd.json_normalize(df['aml']).rename(columns=lambda x: 'aml.' + x)

status = pd.json_normalize(df['status']).rename(columns=lambda x: 'status.' + x)

df

,_id,status,aml,amount,recipient,sender,createdAt,recipient.firstName,recipient.lastName,recipient.bsb,recipient.accountNo,sender.firstName,sender.lastName,sender.accountNo
0,648a55429fd44335887a9451,"{'value': 'CONFIRMED', 'history': [{'value': '...","{'paymentCheck': 648a554543e60d5b82a5dcc7, 're...",300.0,"{'firstName': 'Thomas', 'lastName': 'Wallace',...","{'firstName': 'Thomas', 'lastName': 'Wallace'}",2023-06-15 00:03:14.348,Thomas,Wallace,063097,20391164,Thomas,Wallace,NaN
1,648a56b79fd443d6547a98e8,"{'value': 'CONFIRMED', 'history': [{'value': '...","{'paymentCheck': 648a56ba9fd44305647a9903, 're...",400.0,"{'firstName': 'Paul', 'lastName': 'Sharman', '...","{'firstName': 'Paul', 'lastName': 'Sharman'}",2023-06-15 00:09:27.155,Paul,Sharman,016441,584486701,Paul,Sharman,NaN
2,648a56b743e60d7cefa5e1d2,"{'value': 'CONFIRMED', 'history': [{'value': '...","{'paymentCheck': 648a56bb9fd4430b657a9914, 're...",200.0,"{'firstName': 'Pankaj', 'lastName': 'Kumar', '...","{'firstName': 'Pankaj', 'lastName': 'Kumar'}",2023-06-15 00:09:27.978,Pankaj,Kumar,083004,183408497,Pankaj,Kumar,NaN
3,648a57899fd44305117a9d9d,"{'value': 'CONFIRMED', 'history': [{'value': '...","{'paymentCheck': 648a579043e60d267ba5e674, 're...",1200.0,"{'firstName': 'Salim', 'lastName': 'Farah', 'b...","{'firstName': 'Salim', 'lastName': 'Farah'}",2023-06-15 00:12:57.653,Salim,Farah,037152,218753,Salim,Farah,NaN
4,648a5a0143e60d6821a5f426,"{'value': 'CONFIRMED', 'history': [{'value': '...","{'paymentCheck': 648a5a059fd4434caf7aab6a, 're...",100.0,"{'firstName': 'Hepisipa', 'lastName': 'Uasike'...","{'firstName': 'Hepisipa', 'lastName': 'Uasike'}",2023-06-15 00:23:29.542,Hepisipa,Uasike,063627,10475420,Hepisipa,Uasike,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144491,6561f73fb3dd1028fd5d9f82,"{'value': 'PENDING', 'history': [{'value': 'IN...","{'paymentCheck': 6561f740b3dd10043b5d9f98, 're...",1000.0,"{'firstName': 'Amanda', 'lastName': 'Tetley', ...","{'firstName': 'Amanda', 'lastName': 'Tetley'}",2023-11-25 13:31:43.928,Amanda,Tetley,084687,698528236,Amanda,Tetley,NaN
144492,6561f897b3dd1007fd5da40e,"{'value': 'PENDING', 'history': [{'value': 'IN...","{'paymentCheck': 6561f897b3dd10b5085da423, 're...",2900.0,"{'firstName': 'Nattalie', 'lastName': 'Carr', ...","{'firstName': 'Nattalie', 'lastName': 'Carr'}",2023-11-25 13:37:27.219,Nattalie,Carr,062692,34721873,Nattalie,Carr,NaN
144493,6561f92f8d6bf698db1dc436,"{'value': 'PENDING', 'history': [{'value': 'IN...","{'paymentCheck': 6561f92f8d6bf6cce61dc44b, 're...",150.0,"{'firstName': 'Kayla', 'lastName': 'Robertson'...","{'firstName': 'Kayla', 'lastName': 'Robertson'}",2023-11-25 13:39:59.163,Kayla,Robertson,065903,10543046,Kayla,Robertson,NaN
144494,6561fa3eb3dd10d7cf5dace9,"{'value': 'REVIEWING', 'history': [{'value': '...","{'paymentCheck': 6561fa3eb3dd10335b5dad00, 're...",3806.9,"{'firstName': 'Nicola', 'lastName': 'HEMMING',...","{'firstName': 'Jade', 'lastName': 'Thomson'}",2023-11-25 13:44:30.359,Nicola,HEMMING,013740,155214345,Jade,Thomson,NaN


In [9]:
status = pd.json_normalize(df['status']).rename(columns=lambda x: 'status.' + x)
df = pd.concat([df, status], axis=1)
df.drop(['status'], axis=1, inplace=True)

In [10]:
df.columns

Index(['_id', 'aml', 'amount', 'recipient', 'sender', 'createdAt',
       'recipient.firstName', 'recipient.lastName', 'recipient.bsb',
       'recipient.accountNo', 'sender.firstName', 'sender.lastName',
       'sender.accountNo', 'status.value', 'status.history', 'status.msg'],
      dtype='object')

In [11]:
def has_been_reviewed(x):
    for i in x:
        if i['value'] == 'REVIEWING':
            return True
    return False
        

df['reviewed'] =  df['status.history'].apply(lambda x: has_been_reviewed(x))

In [12]:
df['reviewed'].value_counts(normalize=True)

False    0.820936
True     0.179064
Name: reviewed, dtype: float64

In [13]:
aml = pd.json_normalize(df['aml']).rename(columns=lambda x: 'aml.' + x)
df = pd.concat([df, aml], axis=1)

In [14]:
# merge with payment check
df = pd.merge(df, paymentCheck_df, left_on='aml.paymentCheck', right_on='_id', how='left')

In [15]:
df.drop(['_id_x', 'aml', '_id_y'], axis=1, inplace=True)

In [16]:
df['toActorType'] = pd.json_normalize(df['toLegActor'])['actorType']
df['fromActorType'] = pd.json_normalize(df['fromLegActor'])['actorType']

df.drop(['toLegActor', 'fromLegActor'], axis=1, inplace=True)

In [17]:
# reported true if toLegActor or fromLegActor SUSPICIOUS or BAD
df['reported'] = df['toActorType'].apply(lambda x: x in ['SUSPICIOUS', 'BAD']) | df['fromActorType'].apply(lambda x: x in ['SUSPICIOUS', 'BAD'])

In [18]:
df['reported'].value_counts(normalize=True)

False    0.988276
True     0.011724
Name: reported, dtype: float64

In [19]:
df.to_csv('transactions_before_anonymizations_2.csv', index=False)

In [20]:
recipient = pd.json_normalize(df['recipient']).rename(columns=lambda x: 'recipient.' + x)
df = pd.concat([df, recipient], axis=1)

sender = pd.json_normalize(df['sender']).rename(columns=lambda x: 'sender.' + x)
df = pd.concat([df, sender], axis=1)

In [21]:
name_mapping = {}

# Create a Faker instance
fake = Faker()

# Function to generate a fake name while maintaining consistency
def generate_fake_name(original_name):
    # If the original name is encountered for the first time, generate a new fake name
    if original_name not in name_mapping:
        fake_name = fake.name()
        name_mapping[original_name] = fake_name
    else:
        # Use the existing fake name for consistent mapping
        fake_name = name_mapping[original_name]
    return fake_name

# Iterate through the dataframe and generate fake names
recipient_fake_names = []
sender_fake_names = []
for _, row in df.iterrows():
    first_name = row['recipient.firstName']
    last_name = row['recipient.lastName']
    recipient_full_name = f"{first_name} {last_name}"
    
    fake_name = generate_fake_name(recipient_full_name.lower())
    recipient_fake_names.append(fake_name)
    # to lower
    
    first_name = row['sender.firstName']
    last_name = row['sender.lastName']
    sender_full_name = f"{first_name} {last_name}"
    
    fake_name = generate_fake_name(sender_full_name.lower())
    sender_fake_names.append(fake_name)

# Add the fake names column to the dataframe
df['recipient.fake_name'] = recipient_fake_names
df['sender.fake_name'] = sender_fake_names

In [22]:
df

,amount,recipient,sender,createdAt,recipient.firstName,recipient.lastName,recipient.bsb,recipient.accountNo,sender.firstName,sender.lastName,...,reported,recipient.firstName,recipient.lastName,recipient.bsb,recipient.accountNo,sender.firstName,sender.lastName,sender.accountNo,recipient.fake_name,sender.fake_name
0,300.0,"{'firstName': 'Thomas', 'lastName': 'Wallace',...","{'firstName': 'Thomas', 'lastName': 'Wallace'}",2023-06-15 00:03:14.348,Thomas,Wallace,063097,20391164,Thomas,Wallace,...,False,Thomas,Wallace,063097,20391164,Thomas,Wallace,NaN,Kathryn Gomez,Daniel Tyler
1,400.0,"{'firstName': 'Paul', 'lastName': 'Sharman', '...","{'firstName': 'Paul', 'lastName': 'Sharman'}",2023-06-15 00:09:27.155,Paul,Sharman,016441,584486701,Paul,Sharman,...,False,Paul,Sharman,016441,584486701,Paul,Sharman,NaN,Hailey Castillo,Keith Johnson
2,200.0,"{'firstName': 'Pankaj', 'lastName': 'Kumar', '...","{'firstName': 'Pankaj', 'lastName': 'Kumar'}",2023-06-15 00:09:27.978,Pankaj,Kumar,083004,183408497,Pankaj,Kumar,...,False,Pankaj,Kumar,083004,183408497,Pankaj,Kumar,NaN,Adam Mckinney,Erica Reed
3,1200.0,"{'firstName': 'Salim', 'lastName': 'Farah', 'b...","{'firstName': 'Salim', 'lastName': 'Farah'}",2023-06-15 00:12:57.653,Salim,Farah,037152,218753,Salim,Farah,...,False,Salim,Farah,037152,218753,Salim,Farah,NaN,James Stone,Michelle Dickerson
4,100.0,"{'firstName': 'Hepisipa', 'lastName': 'Uasike'...","{'firstName': 'Hepisipa', 'lastName': 'Uasike'}",2023-06-15 00:23:29.542,Hepisipa,Uasike,063627,10475420,Hepisipa,Uasike,...,False,Hepisipa,Uasike,063627,10475420,Hepisipa,Uasike,NaN,Sandra Watson,Paul Singleton
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144491,1000.0,"{'firstName': 'Amanda', 'lastName': 'Tetley', ...","{'firstName': 'Amanda', 'lastName': 'Tetley'}",2023-11-25 13:31:43.928,Amanda,Tetley,084687,698528236,Amanda,Tetley,...,False,Amanda,Tetley,084687,698528236,Amanda,Tetley,NaN,Jake Hill,Ronald Brown
144492,2900.0,"{'firstName': 'Nattalie', 'lastName': 'Carr', ...","{'firstName': 'Nattalie', 'lastName': 'Carr'}",2023-11-25 13:37:27.219,Nattalie,Carr,062692,34721873,Nattalie,Carr,...,False,Nattalie,Carr,062692,34721873,Nattalie,Carr,NaN,Cristian Mejia,Richard Wang
144493,150.0,"{'firstName': 'Kayla', 'lastName': 'Robertson'...","{'firstName': 'Kayla', 'lastName': 'Robertson'}",2023-11-25 13:39:59.163,Kayla,Robertson,065903,10543046,Kayla,Robertson,...,False,Kayla,Robertson,065903,10543046,Kayla,Robertson,NaN,Tina Rose,Marissa Fisher
144494,3806.9,"{'firstName': 'Nicola', 'lastName': 'HEMMING',...","{'firstName': 'Jade', 'lastName': 'Thomson'}",2023-11-25 13:44:30.359,Nicola,HEMMING,013740,155214345,Jade,Thomson,...,False,Nicola,HEMMING,013740,155214345,Jade,Thomson,NaN,Matthew Gibson,Adam Adams


In [36]:
# take only one column with same name
print(df['recipient.accountNo'].iloc[:, 0])

0          20391164
1         584486701
2         183408497
3            218753
4          10475420
            ...    
144491    698528236
144492     34721873
144493     10543046
144494    155214345
144495    537853206
Name: recipient.accountNo, Length: 144496, dtype: object


In [37]:
df['recipient.accountNumber'] = df['recipient.bsb'].iloc[:, 0].astype(str) + '' +df['recipient.accountNo'].iloc[:, 0].astype(str)
df['recipient.accountNumber']

0          06309720391164
1         016441584486701
2         083004183408497
3            037152218753
4          06362710475420
               ...       
144491    084687698528236
144492     06269234721873
144493     06590310543046
144494    013740155214345
144495    014504537853206
Name: recipient.accountNumber, Length: 144496, dtype: object

In [38]:
account_mapping = {}

# Function to hash an account number using hashlib
def hash_account_number(account_number):
    if account_number in account_mapping:
        hashed_account = account_mapping[account_number]
    else:
        sha256 = hashlib.sha256()
        sha256.update(account_number.encode())
        hashed_account = sha256.hexdigest()
        account_mapping[account_number] = hashed_account
    return hashed_account

hashed_account_numbers = []
for _, row in df.iterrows():
    original_account_number = row['recipient.accountNumber']
    hashed_account = hash_account_number(original_account_number)
    hashed_account_numbers.append(hashed_account)

# Replace the original account numbers with hashed values
df['recipient.fake_accountNumber'] = hashed_account_numbers

In [39]:
df['recipient.fake_accountNumber'] = df['recipient.fake_accountNumber'].apply(lambda x: x[:20])

In [40]:
def anonymize_amount(original_amount, percentage_range=(0.0, 0.2)):
    min_percentage, max_percentage = percentage_range
    random_percentage = random.uniform(min_percentage, max_percentage)
    
    change_amount = original_amount * random_percentage
    sign = random.choice([-1, 1])
    
    anonymized_amount = original_amount + sign * change_amount
    return anonymized_amount

# Anonymize the 'amount' column
anonymized_amounts = []
for _, row in df.iterrows():
    original_amount = row['amount']
    anonymized_amount = anonymize_amount(original_amount)
    anonymized_amounts.append(anonymized_amount)

# Replace the original 'amount' values with anonymized values
df['fake_amount'] = anonymized_amounts

In [41]:
df['fake_amount'] = df['fake_amount'].round(2)

In [42]:
# in createAt keep only day month and year remove time
df['createdAt'] = pd.to_datetime(df['createdAt']).dt.strftime('%Y-%m-%d')
df['createdAt']

0         2023-06-15
1         2023-06-15
2         2023-06-15
3         2023-06-15
4         2023-06-15
             ...    
144491    2023-11-25
144492    2023-11-25
144493    2023-11-25
144494    2023-11-25
144495    2023-11-25
Name: createdAt, Length: 144496, dtype: object

In [43]:
df.rename(columns={'fake_amount': 'tx_amount', 'recipient.fake_name': 'recipient_full_name', 'sender.fake_name': 'sender_full_name',
                   'recipient.fake_accountNumber': 'recipient_accountNumber'}, inplace=True)

In [44]:
df[['tx_amount', 'createdAt', 'recipient_accountNumber', 'recipient_full_name', 'sender_full_name', 'reviewed', 'reported' ]]

,tx_amount,createdAt,recipient_accountNumber,recipient_full_name,sender_full_name,reviewed,reported
0,315.13,2023-06-15,18bc5486b344275c509b,Kathryn Gomez,Daniel Tyler,True,False
1,426.33,2023-06-15,6f0ed9c6d1380cf32123,Hailey Castillo,Keith Johnson,False,False
2,217.22,2023-06-15,503bba37df5ffc72eea0,Adam Mckinney,Erica Reed,True,False
3,1063.67,2023-06-15,4a8bb17b8f2b28ddaffc,James Stone,Michelle Dickerson,True,False
4,89.78,2023-06-15,242d39ff5a44321d4f6f,Sandra Watson,Paul Singleton,True,False
...,...,...,...,...,...,...,...
144491,926.99,2023-11-25,b27175d0654d322ddee9,Jake Hill,Ronald Brown,False,False
144492,3418.03,2023-11-25,f99d79908c935b5a04a5,Cristian Mejia,Richard Wang,False,False
144493,142.75,2023-11-25,6dd8951dcd16b38e4dcc,Tina Rose,Marissa Fisher,False,False
144494,4438.86,2023-11-25,d9d2708900afc4df1251,Matthew Gibson,Adam Adams,True,False


In [46]:
df[['tx_amount', 'createdAt', 'recipient_accountNumber', 'recipient_full_name', 'sender_full_name', 'reviewed', 'reported' ]]

,tx_amount,createdAt,recipient_accountNumber,recipient_full_name,sender_full_name,reviewed,reported
0,315.13,2023-06-15,18bc5486b344275c509b,Kathryn Gomez,Daniel Tyler,True,False
1,426.33,2023-06-15,6f0ed9c6d1380cf32123,Hailey Castillo,Keith Johnson,False,False
2,217.22,2023-06-15,503bba37df5ffc72eea0,Adam Mckinney,Erica Reed,True,False
3,1063.67,2023-06-15,4a8bb17b8f2b28ddaffc,James Stone,Michelle Dickerson,True,False
4,89.78,2023-06-15,242d39ff5a44321d4f6f,Sandra Watson,Paul Singleton,True,False
...,...,...,...,...,...,...,...
144491,926.99,2023-11-25,b27175d0654d322ddee9,Jake Hill,Ronald Brown,False,False
144492,3418.03,2023-11-25,f99d79908c935b5a04a5,Cristian Mejia,Richard Wang,False,False
144493,142.75,2023-11-25,6dd8951dcd16b38e4dcc,Tina Rose,Marissa Fisher,False,False
144494,4438.86,2023-11-25,d9d2708900afc4df1251,Matthew Gibson,Adam Adams,True,False


In [45]:
df[['tx_amount', 'createdAt', 'recipient_accountNumber', 'recipient_full_name', 'sender_full_name', 'reviewed', 'reported' ]].to_csv('transactions_after_anonymizations.csv', index=False)

In [53]:
df['reported'].value_counts(normalize=True)

False    0.981496
True     0.018504
Name: reported, dtype: float64